# Notebook de versement de données

Ce notebook permet de verser des données Excel dans une base PostgreSQL.

## 1. Importation des packages

In [ ]:
import sys
sys.path.append('..')

from script.body.base_donnees import ConnectionBaseDeDonnees
from script.body.livre import Livre
from script.body.traitement import Traitement
from script.body.versement import Versement
from script.branch.gestion_dossier import GestionDossier
from script.leaf.validator import DataValidator
from script.leaf.catalogue import Catalogue
from script.branch.gomme import Gomme

## 2. Configuration de la connexion

In [ ]:
# Connexion à la base de données
db = ConnectionBaseDeDonnees()
conn, schema = db.connexion_observatoire()

## 3. Configuration du versement

In [ ]:
# Paramètres du versement
fichier_excel = "../data/hola.xlsx"
annee = 2025
theme = "NOMTHEME"
base = "NOMBASE"
source = "NOMSOURCE"

# Validation du fichier
if not DataValidator.validate_excel_file(fichier_excel):
    raise ValueError("Fichier Excel invalide.")
else:
    print("Fichier Excel valide.")

# Enregistrement des paramètres du versement
livre = Livre(
    conn, 
    schema=schema, 
    theme=theme,
    base=base, 
    source=source, 
    annee=annee
)
livre.echelle = livre.choix_echelle()

## Catalogue / Facultatif

In [ ]:
catalogue = Catalogue(livre)
catalogue.liste_table(afficher=True)

In [ ]:
catalogue.catalogue()

## 4. Traitement des données

In [ ]:
# Traitement des données
traiteur = Traitement(conn, fichier_excel, livre)
traiteur.traitement()

## 5. Sauvegarde en base de données

In [ ]:
# Sauvegarde des données
versement = Versement(conn, livre)
versement.versement()

## 6. Suppression des fichiers temporaires

In [ ]:
gestionnaire_de_dossier = GestionDossier(livre)
gestionnaire_de_dossier.delete_folder()

## 7. Suppression de table / versement

In [ ]:
gomme = Gomme(conn, livre)
gomme.suppression_table(table_cible='', id_versement_cible=3, garder_table=False)

## 8. Ajout des contraintes sur les tables en production

In [ ]:
# Connexion à la base de données
db = ConnectionBaseDeDonnees()
conn, schema = db.connexion_observatoire()

theme = "NOMTHEME"
base = "NOMBASE"
source = "NOMSOURCE"
annee=2025

livre = Livre(
    conn, 
    schema=schema, 
    theme=theme,
    base=base, 
    source=source, 
    annee=annee
)

In [ ]:
livre.relations = {'primaire': {'contrainte_table_valeur': 'id_composite',
  'contrainte_test_mod': 'id_mod',
  'contrainte_test_var': 'id_var',
  'contrainte_test_vers': 'id_versement'},
 'etrangere': {'contrainte_test_mod': ('id_mod', 'id_mod'),
  'contrainte_test_var': ('id_var', 'id_var'),
  'contrainte_test_vers': ('id_versement', 'id_versement')}}

In [ ]:
versement = Versement(conn, livre)
tables = [
    'contrainte_table_valeur', # livre.nom_table,
    # 'contrainte_test_var', # livre.nom_table_var,
    # 'contrainte_test_mod', # livre.nom_table_mod,
    # 'contrainte_test_vers', # livre.nom_table_vers,
]
print(schema)
for nom in tables:
    print(nom)
versement.ajout_contraintes(schema, tables)